In [12]:
import os
import numpy as np

The values for the fragmentation functions (FF) are taken from the publication:
https://journals.aps.org/prd/abstract/10.1103/PhysRevD.105.L011103
The branching ratios for the decays into electrons are taken from the 2020 ed. of the PDG booklet.
For most charmed hadrons inclusive branching ratios are provided, however in the case of $\Xi_{c}^{0}$ and $\Xi_{c}^{+}$ only one deacy in electrons is listed and no inclusive value is given.

The values are stored in a nested list.
Each sublist is contains: `name, FF,statFF,syst up, syst down, BR, unc up, unc down`

In [13]:
valuesHadrons = [["$D^{0}$",  39.1 ,1.7, 2.5, 3.7,  6.49, 0.11, 0.11],
["$D^{+}$",                   17.3 ,1.8, 1.7, 2.1, 16.07, 0.30, 0.30],
["$D^{+}_{\mathrm s}$",       7.3  ,1.0, 1.9, 1.1,  6.50, 0.40, 0.40],
["$\Lambda^{+}_{\mathrm c}$", 20.4 ,1.3, 1.6, 2.2,  3.95, 0.35, 0.35],
["$\Xi^{0}_{\mathrm c}$",     8.0  ,1.2, 2.5, 2.4,  1.80, 1.20, 1.20],
["$\Xi^{+}_{\mathrm c}$",     8.0  ,1.2, 2.5, 2.4,  2.30, 0.70, 0.80]]

The central value is simply the product of the fragmentation function which is `[1]` and the branching ratio `[5]`.
The uncertainties then follow:
* [2] : statistical uncertainty of FF
* [3] : upper systematic uncertainty of FF
* [4] : lower systematic uncertainty of FF
* [6] : upper uncertainty of BR (from PDG)
* [7] : upper uncertainty of BR (from PDG)
To make the further calculations easier, we also add the stat. and syst. in the FF case.

In [29]:
def quadSum (A,B):
    return np.sqrt(A**2 + B**2)

for i in range(len(valuesHadrons)):
    print(quadSum(valuesHadrons[i][2],valuesHadrons[i][3]),quadSum(valuesHadrons[i][2],valuesHadrons[i][4]))

3.023243291566195 4.071854614300467
2.4758836806279896 2.7658633371878665
2.1470910553583886 1.4866068747318506
2.0615528128088303 2.5553864678361276
2.773084924772409 2.6832815729997477
2.773084924772409 2.6832815729997477


With this we can simplify the list and the make the calculations a bit more readable.

In [52]:
valuesHadrons2 = [["$D^{0}$", 39.1 ,3.0232, 4.0719,  6.49, 0.11, 0.11],
["$D^{+}$",                   17.3 ,2.4759, 2.7659, 16.07, 0.30, 0.30],
["$D^{+}_{\mathrm s}$",       7.3  ,2.1471, 1.4866,  6.50, 0.40, 0.40],
["$\Lambda^{+}_{\mathrm c}$", 20.4 ,2.0616, 2.5554,  3.95, 0.35, 0.35],
["$\Xi^{0}_{\mathrm c}$",     8.0  ,2.7731, 2.6833,  1.80, 1.20, 1.20],
["$\Xi^{+}_{\mathrm c}$",     8.0  ,2.7731, 2.6833,  2.30, 0.70, 0.80]]

In [60]:
def calcFtBR (list_):
    val = (list_[1]/100.)*(list_[4]/100)
    # gaussian error propagation:
    # A * B -> Add the relative uncertainties
    # they are uncorrelated -> quadratic sum
    un_u = np.sqrt( (list_[2]/list_[1])**2 + (list_[5]/list_[4])**2 )
    un_l = np.sqrt( (list_[3]/list_[1])**2 + (list_[6]/list_[4])**2 )
    # back to %
    val *= 100
    # and back to absolute uncertainties
    un_u *= val
    un_l *= val
    # return a list with the 
    return [list_[0], val, un_u, un_l]

In [70]:
c2e = 0.
c2e_u = 0.
c2e_l = 0.
for i in range(len(valuesHadrons2)):
    l = calcFtBR(valuesHadrons2[i])
#     print(l)
    c2e += l[1]
    c2e_u += (l[2]**2)
    c2e_l += (l[3]**2)
c2e_u = np.sqrt(c2e_u)
c2e_l = np.sqrt(c2e_l)
print("c2e : ",c2e, "+",np.sqrt(c2e_u),"-",np.sqrt(c2e_l))
print("c2e : ",c2e, "+",c2e_u/c2e*100,"%-",c2e_l/c2e*100,"%")


c2e :  6.926 + 0.7087397433129405 - 0.7502762781577206
c2e :  6.926 + 7.25255593057021 %- 8.127555494747348 %


Now we can calculate the $c\bar{c} \rightarrow ee$ as c2e$^2$. For the uncertainties we solve:
\begin{equation}
\delta f(x,\delta x) := \sqrt{\frac{\partial f(x)}{\partial x}\delta x} 
\end{equation}
with $x^2$ we can wite:
\begin{equation}
\delta f(x,\delta x) := \sqrt{2 \cdot x \cdot \delta x} 
\end{equation}

In [85]:
cc2ee = [c2e**2/100, np.sqrt(2* c2e * c2e_u)/100,  np.sqrt(2* c2e * c2e_l)/100]
print(cc2ee)

[0.47969476, 0.02637807072741088, 0.027923988907172667]


This should now be compared with the value we have from the $Z^0 \rightarrow ee$ of $9.6\pm0.4$%.



In [86]:
cc2ee_old = [9.6**2/100, np.sqrt(2* 9.6 * 0.4)/100,  np.sqrt(2* 9.6 * 0.4)/100]
print(cc2ee_old)

[0.9216, 0.027712812921102038, 0.027712812921102038]


In [87]:
print("The new points will be a factor", cc2ee_old[0]/cc2ee[0] ,"higher")

The new points will be a factor 1.9212217369228715 higher


In [84]:
print("The uncertainty is now ",cc2ee[2]/cc2ee[0],"instead of 22%")

The uncertainty is now  0.05821199486767933 instead of 22%
